### Agents, Tools and Routing
In this Notebook we explore how to define our own functions and tools and employ routing to call these functions.
We will use both the Wikipedia function and a News api function to call different sources of information.

In [88]:
import os
import openai
from datetime import datetime, date, timedelta
# from newspaper import Article
from config import API_KEY
from newsapi  import NewsApiClient
import zipfile
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.agents import tool
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities import StackExchangeAPIWrapper

### Defining the Wikipedia tool
This tool allows you to perform a general search on wikipedia 

In [45]:
# Wikipedia Tool
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

### Defining the News API tool
This tool allows you to call the newspaw api and get up to date news on a topic of your choice

In [84]:
# News API tool
import requests
from pydantic import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function

@tool
def get_news_articles(query: str) -> str:
    """Run news articles search and get summaries"""
    
    print(query)
    # Init
    newsapi = NewsApiClient(api_key=API_KEY)
    
  
    date_to = date.today()


    date_from = date_to - timedelta(days=2)
    date_to = datetime.strftime(date_to,'%Y-%m-%d')
    date_from = datetime.strftime(date_from,'%Y-%m-%d')
    print("Getting for dates from", date_from, "to", date_to)
    # get everything using topic for query key
    # retrieve only 10 articles
    articles= newsapi.get_everything(q=query,language='en', page=1,page_size=10, from_param=date_from,
                                      to=date_to)

    art=[]
    
    for i in range(len(articles['articles'])):
        title = articles['articles'][i]['title']
        url = articles['articles'][i]['url']
        blurp = articles['articles'][i]['description']
        imgurl = articles['articles'][i]['urlToImage']
        article_date = articles['articles'][i]['publishedAt']
        article_date = article_date.split('T')[0]
        source = articles['articles'][i]['source']['name'].lower()

        try:
            text, blurp1 = get_key_word(url)
        except:
            text = articles['articles'][i]['content']

        try:
            article_date = datetime.strptime(article_date, "%Y-%m-%d")
            article_date = datetime.strftime(article_date, "%d/%m/%Y")
        except IndexError:
            article_date = date.today().replace(day=1)
            article_date = datetime.strftime(article_date, "%d/%m/%Y")
        art.append(
        {
            'title': title,
            'imgurl': imgurl,
            'date': article_date,
            'blurp': blurp,
            'url': url,
            'text': text,
            'source': source
        })
    return art


In [85]:
# test the function
get_news_articles('artificial intelligence')

artificial intelligence
Getting for dates from 2024-11-20 to 2024-11-22


[{'title': 'Must-do privacy settings on your iPhone in iOS 18.1',
  'imgurl': 'https://static.foxnews.com/foxnews.com/content/uploads/2024/11/1-must-turn-off-privacy-settings-on-your-iphone-in-ios-18.1-ai.jpg',
  'date': '20/11/2024',
  'blurp': "Apple's latest iOS update introduces advanced artificial intelligence capabilities, which may be capturing and analyzing sensitive information.",
  'url': 'https://www.foxnews.com/tech/must-do-privacy-settings-your-iphone-ios-18-1',
  'text': 'With Apple rolling out the latest iOS 18.1 update, privacy has become an even hotter topic. The update introduces advanced AI capabilities through Apple Intelligence and Siri, giving your device more… [+6131 chars]',
  'source': 'fox news'},
 {'title': 'A Trump 2.0 Agenda For Artificial Intelligence',
  'imgurl': 'https://imageio.forbes.com/specials-images/imageserve/673e07e448f52276f6a8628d/0x0.jpg?format=jpg&height=900&width=1600&fit=bounds',
  'date': '20/11/2024',
  'blurp': 'As Trump begins his seco

In [23]:
get_news_articles.name

'get_news_articles'

In [86]:
get_news_articles({"query":"langchain"})

langchain
Getting for dates from 2024-11-23 to 2024-11-25


[{'title': '32k context length text embedding models',
  'imgurl': 'https://i0.wp.com/blog.voyageai.com/wp-content/uploads/2024/09/voyage-3-vector-space.jpg?fit=1024%2C1024&quality=89&ssl=1',
  'date': '24/11/2024',
  'blurp': 'TL;DR – We are excited to announce voyage-3 and voyage-3-lite embedding models, advancing the frontier of retrieval quality, latency, and cost. voyage-3 outperforms OpenAI v3 large by 7.55% on aver…',
  'url': 'https://blog.voyageai.com/2024/09/18/voyage-3/',
  'text': 'TL;DR We are excited to announce voyage-3 and voyage-3-lite embedding models, advancing the frontier of retrieval quality, latency, and cost. voyage-3 outperforms OpenAI v3 large by 7.55% on average … [+8419 chars]',
  'source': 'voyageai.com'},
 {'title': 'Dear IT departments stop trying to build your own RAG',
  'imgurl': 'https://miro.medium.com/v2/resize:fit:940/1*8yJrgGPFwHBY8lpIFg7ERQ.png',
  'date': '24/11/2024',
  'blurp': 'You would never ever in a million years build your own CRM system

In [28]:
from langchain.tools.render import format_tool_to_openai_function

In [29]:
get_news_articles_fn =format_tool_to_openai_function(get_news_articles)

In [32]:
get_news_articles_fn

{'name': 'get_news_articles',
 'description': 'Run news articles search and get summaries',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [33]:
model = ChatOpenAI(temperature=0).bind(functions=[get_news_articles_fn])

In [39]:
model.invoke("get 3 articles on langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"langchain"}', 'name': 'get_news_articles'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-912b272f-a6cd-48ec-baaf-3aebe1673d53-0')

In [40]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [41]:
result = chain.invoke({"input": "what articles write about langchain?"})

In [42]:
result

AgentActionMessageLog(tool='get_news_articles', tool_input={'query': 'Langchain'}, log="\nInvoking: `get_news_articles` with `{'query': 'Langchain'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'get_news_articles'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 62, 'total_tokens': 78, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-852d73fb-873f-4694-acda-379074f3bac2-0')])

### Defining a tool to call stackexchange 
Stackexchange is a network of question-and-answer (Q&A) websites each site covering a specific topic, where questions, answers, and users are subject to a reputation award process. You can use this to access stackoverflow for coding problems

In [98]:
@tool
def get_stackexchange_solution(query: str) -> str:
    "Stackexchange is a network of question-and-answer (Q&A) websites on topics in diverse fields, each site covering a specific topic, where questions, answers, and users are subject to a reputation award process. It can be used to access stackoverflow which focuses on computer programming"
    stackexchange = StackExchangeAPIWrapper()
    results = stackexchange.run(query)
    return results

In [99]:

print(get_stackexchange_solution("what is AttributeError"))

Question: Unable to access variable that has been declared inside a class, outside of it
My code <span class="highlight">is</span> following:
class makequestions(QMainWindow):

    def __init__(self):
        super(makequestions, self). &hellip; : &#39;makequestions&#39; object has no attribute &#39;questions&#39;

Edit: I tried applying <span class="highlight">what</span> @buzz8year advised, - declaring questions class property, - it works but it ends up taking the value of questions as None &hellip; 

Question: How to get information out of &#39;OneToOneField&#39; and use this information in admin.py list_display?
I making my first site, this <span class="highlight">is</span> my first big project so I stuck at one issue and can&#39;t solve it.. &hellip; I said <span class="highlight">what</span> I tried above. &hellip; 

Question: Problem when trying to get Sector Market Cap
Market Cap through the yfinance API through the following code:
import yfinance as yf
tech = yf.Sector(&#39;te

In [100]:

functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_news_articles, get_stackexchange_solution
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

### Routing using Agent
- Using routing we can either access the wikipedia search, obtain news articles related to the topic we are interested in for more relevant news, or search for problems within stack exchange
- As compared to the previous section, routing outputs the result of the tool instead of just the output of the AI agent

In [101]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_news_articles": get_news_articles,
            "get_stackexchange_solution": get_stackexchange_solution
        }
        return tools[result.tool].run(result.tool_input)

In [102]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [103]:
result = chain.invoke({"input": "How do I multiply each element in a list by a number"})

In [104]:
result

"You can multiply each element in a list by a number in Python using list comprehension. Here's an example code snippet:\n\n```python\n# Define a list\nmy_list = [1, 2, 3, 4, 5]\n\n# Define a number to multiply each element by\nmultiplier = 2\n\n# Multiply each element in the list by the multiplier using list comprehension\nresult = [element * multiplier for element in my_list]\n\nprint(result)\n```\n\nIn this code snippet, `my_list` is the list of elements, and `multiplier` is the number you want to multiply each element by. The list comprehension `[element * multiplier for element in my_list]` multiplies each element in `my_list` by `multiplier` and stores the result in the `result` list.\n\nFeel free to try this code snippet in your Python environment!"

In [105]:
result = chain.invoke({"input": "What articles talk about langchain?"})

langchain
Getting for dates from 2024-11-26 to 2024-11-28


In [108]:
result

[{'title': 'Anthropic releases Model Context Protocol to standardize AI-data integration',
  'imgurl': 'https://venturebeat.com/wp-content/uploads/2023/12/AdobeStock_621371321.jpeg?w=1024?w=1200&strip=all',
  'date': '26/11/2024',
  'blurp': 'Model Context Protocol, a new open source release from Anthropic, aims to eliminate the need to write code for every AI data integration.',
  'url': 'https://venturebeat.com/data-infrastructure/anthropic-releases-model-context-protocol-to-standardize-ai-data-integration/',
  'text': 'Join our daily and weekly newsletters for the latest updates and exclusive content on industry-leading AI coverage. Learn More\r\nOne decision many enterprises have to make when implementing AI use cas… [+3736 chars]',
  'source': 'venturebeat'},
 {'title': 'Crafting a hybrid geospatial RAG application with Elastic and Amazon Bedrock',
  'imgurl': 'https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt4e527c4d30fc4d9d/6745ecd1278118edb95d6058/power-of-your-da